# Optimisation of commerical building energy system

ToDo: provide context of decarbonisation task, building energy system model, etc.

#### Setup

In [1]:
# import packages and required functions
from tqdm.auto import tqdm
import numpy as np
from numpy import random
from scipy import stats
from models import run_model
from functools import partial
from utils import get_Gurobi_WLS_env, fmt_design_results

In [2]:
# quality of life stuff
env = get_Gurobi_WLS_env(silence=True)
run_model = partial(run_model, env=env)

In [3]:
solar_years = range(2012,2018)
load_years = range(2012,2018)
random.seed(42)

#### ... hand-fed design ...

In [4]:
design_options = [
    [500,400],
    [500,600],
    [600,600],
    [750,600],
    [750,800],
]

In [ ]:
det_results = [run_model(*design) for design in tqdm(design_options, desc='Assessing designs')]

In [ ]:
print([r['total'] for r in det_results])

In [ ]:
best_det_design = det_results[np.argmin([r['total'] for r in det_results])]
print(fmt_design_results(best_det_design))

In [ ]:
nosystem_results = run_model(0, 0)
print(fmt_design_results(nosystem_results))

Therefore, installing the solar-battery system saves ..., i.e. reduces total cost of providing building load by ...%

#### Designing using the Linear Program

In [ ]:
LP_results = run_model(None,None)
print(fmt_design_results(LP_results))

Improvement from LP vs static options

In [ ]:
design_improvement = best_det_design['total'] - LP_results['total']
print(f"£{round((design_improvement)/1e3,2)}k/yr")
print(f"{round((design_improvement)/best_det_design['total']*100,2)}%")

#### Accounting for uncertainty during design

In [ ]:
stoch_results = [run_model(*design, solar_year=solar_years) for design in tqdm(design_options, desc='Assessing designs')]
# note trick of getting optimiser to simultaneously solve independent scenarios

In [ ]:
best_stoch_design = stoch_results[np.argmin([r['total'] for r in stoch_results])]
print(fmt_design_results(best_stoch_design))

Look at cost estimation error of deterministic optimisation

In [ ]:
cost_est_error = best_stoch_design['total'] - best_det_design['total']
print(f"£{round((cost_est_error)/1e3,2)}k/yr")
print(f"{round((cost_est_error)/best_stoch_design['total']*100,2)}%")

SP ...

In [ ]:
SP_results = run_model(None,None,solar_year=list(range(2012,2018)))
print(fmt_design_results(SP_results))

Improvement from SP vs static options

In [ ]:
design_improvement = best_stoch_design['total'] - SP_results['total']
print(f"£{round((design_improvement)/1e3,2)}k/yr")
print(f"{round((design_improvement)/best_stoch_design['total']*100,2)}%")

... VSS ...

In [ ]:
try:
    run_model(LP_results['solar_capacity'],LP_results['battery_capacity'],solar_year=[2017])
except Exception as e:
    print(f"Encountered error: {e}")

Oops, deterministic LP leads to an infeasible solution given the uncertainties!

In [ ]:
LP_avg_cost = run_model(
    LP_results['solar_capacity']*0.95,
    LP_results['battery_capacity'],
    solar_year=solar_years
    )['total']

vss = (LP_avg_cost - SP_results['total'])
print(f"Value of stochastic solution: £{round((vss)/1e3,2)}k/yr")
print(f"VSS as % of SP solution: {round((vss)/SP_results['total']*100,2)}%")

Stochastic optimisation with lots of uncertainties ...

In [ ]:
nsamples = 10
scenarios = {
    'solar_year': random.choice(solar_years, nsamples),
    'load_year': random.choice(load_years, nsamples),
    'mean_load': stats.truncnorm.rvs(-2, 2, loc=100, scale=10, size=nsamples),
    'battery_efficiency': stats.truncnorm.rvs(-2, 2, loc=0.95, scale=0.05, size=nsamples),
    'battery_cost': stats.truncnorm.rvs(-2, 2, loc=70, scale=5, size=nsamples),
}

In [ ]:
stoch2_results = [run_model(*design, **scenarios) for design in tqdm(design_options, desc='Assessing designs')]

In [ ]:
best_stoch2_design = stoch2_results[np.argmin([r['total'] for r in stoch2_results])]
print(fmt_design_results(best_stoch2_design))

In [ ]:
SP_results2 = run_model(
    solar_capacity=None,
    battery_capacity=None,
    solar_year=scenarios['solar_year'],
    load_year=scenarios['load_year'],
    mean_load=scenarios['mean_load'],
    battery_efficiency=scenarios['battery_efficiency'],
    battery_cost=scenarios['battery_cost']
)

In [ ]:
print(fmt_design_results(SP_results2))

Could also then look at benefit of continuous solution (i.e. SP vs discrete stochastic), and Value of Stochastic Solution

In [ ]:
design_improvement = best_stoch2_design['total'] - SP_results2['total']
print(f"£{round((design_improvement)/1e3,2)}k/yr")
print(f"{round((design_improvement)/best_stoch2_design['total']*100,2)}%")

In [ ]:
LP_avg_cost2 = run_model(
    LP_results['solar_capacity']*0.9,
    LP_results['battery_capacity'],
    **scenarios
    )['total']

vss2 = (LP_avg_cost2 - SP_results2['total'])
print(f"Value of stochastic solution: £{round((vss2)/1e3,2)}k/yr")
print(f"VSS as % of SP solution: {round((vss2)/SP_results2['total']*100,2)}%")